In [34]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# Read file "hw1_data.tsv"
file = open("/content/drive/MyDrive/Colab Notebooks/hw1_data.tsv",'r')
ori_data = file.read().strip().split("\n")
data = []
for item in ori_data:
  item = item.split("\t")
  if len(item[0]) >0:
    data.append((item[0],item[1]))
print(len(data))

300


In [36]:
### Training data pre-processing
texts, labels = [], []
label2idx = {"0":0, "1":1}
for i, item in enumerate(data):
  text = item[0]

  ## Preprocessing (if you want to add, please add more)
  ################################################################################
  text = text.replace("  "," ") ## Replace double space
  text = text.replace(",", "") ## Replace comma to ""
  text = text.lower()  ## Lower cases
  ################################################################################
  label = label2idx[item[1]]

  texts.append(text)
  labels.append(label)

print("*"*50)
print("Total number of datasets")
print(len(texts))
print(len(labels))
print("*"*50)




**************************************************
Total number of datasets
300
300
**************************************************


In [37]:
#### Split into train/dev/test sets
from sklearn.model_selection import train_test_split
### Write a code for collecting samples for each class
################################################################################
pos,neg = [], []
train_texts, dev_texts, test_texts, train_labels, dev_labels, test_labels = [], [], [], [], [], []
for a,b in zip(texts,labels):
  if b == 1:
    pos.append((a,b))
  else:
    neg.append((a,b))

for a,b in pos[:50]:
  test_texts.append(a)
  test_labels.append(b)
for a,b in neg[:50]:
  test_texts.append(a)
  test_labels.append(b)

for a,b in pos[50:100]:
  dev_texts.append(a)
  dev_labels.append(b)
for a,b in neg[50:100]:
  dev_texts.append(a)
  dev_labels.append(b)

for a,b in pos[100:]:
  train_texts.append(a)
  train_labels.append(b)
for a,b in neg[100:]:
  train_texts.append(a)
  train_labels.append(b)
################################################################################


tmp = list(zip(train_texts,train_labels))
import random
random.shuffle(tmp)
train_texts, train_labels = zip(*tmp)

tmp = list(zip(dev_texts,dev_labels))
import random
random.shuffle(tmp)
dev_texts, dev_labels = zip(*tmp)

tmp = list(zip(test_texts,test_labels))
import random
random.shuffle(tmp)
test_texts, test_labels = zip(*tmp)

print("Train Dataset Examples")
print(train_texts[:3])
print(train_labels[:3])
print("*"*50)
print(train_labels)
print(dev_labels)

Train Dataset Examples
('handsome but unfulfilling suspense drama ', "the rock 's fighting skills are more in line with steven seagal ", "as a dentist 's waiting room ")
(0, 1, 0)
**************************************************
(0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1)
(0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0)


In [38]:
## Construct a vocabulary
from collections import Counter

all_words = []
for item in train_texts:
  all_words += item.split()
for item in dev_texts:
  all_words += item.split()

## Build a dictionary that maps words to integers
counts = Counter(all_words)
vocab = sorted(counts, key=counts.get, reverse=True)
vocab_to_int = {'<pad>':0, "<unk>":1}
vocab_to_int.update({word: ii for ii, word in enumerate(vocab,2)})
print(vocab_to_int)
print(len(vocab_to_int))

{'<pad>': 0, '<unk>': 1, 'the': 2, 'and': 3, 'a': 4, '.': 5, 'to': 6, 'of': 7, 'it': 8, 'in': 9, 'that': 10, 'is': 11, "'s": 12, 'as': 13, 'for': 14, 'with': 15, 'film': 16, 'you': 17, 'one': 18, 'movie': 19, 'by': 20, 'this': 21, 'its': 22, '--': 23, 'have': 24, '...': 25, 'good': 26, 'so': 27, 'story': 28, 'more': 29, 'no': 30, 'not': 31, 'time': 32, 'if': 33, 'your': 34, "n't": 35, 'most': 36, 'but': 37, 'are': 38, 'way': 39, '(': 40, ')': 41, 'an': 42, 'very': 43, 'little': 44, 'about': 45, 'on': 46, 'i': 47, 'takes': 48, 'own': 49, 'be': 50, 'too': 51, 'worst': 52, '``': 53, 'picture': 54, 'or': 55, 'from': 56, 'some': 57, 'at': 58, 'just': 59, 'can': 60, 'could': 61, 'when': 62, 'comedy': 63, 'his': 64, 'their': 65, 'every': 66, 'like': 67, 'does': 68, 'into': 69, 'all': 70, 'recent': 71, "''": 72, 'many': 73, 'hate': 74, 'life': 75, 'surprisingly': 76, 'watching': 77, 'what': 78, 'better': 79, 'moments': 80, 'old': 81, 'men': 82, 'work': 83, 'completely': 84, 'suspense': 85, 'ro

In [39]:
import numpy as np
def encode_sentence(sentence):
    max_length = 50
    input_ids = []
    for item in sentence.split():
      if item in vocab_to_int:
        input_ids.append(vocab_to_int[item])
      else:
        input_ids.append(vocab_to_int['<unk>'])

    padding_length = max_length - len(input_ids)
    input_ids += [vocab_to_int['<pad>']] * padding_length
    return np.array(input_ids)

def encode_label(label):
    return np.array(label)



print("Train Dataset Encode Examples")
for a,b in zip(train_texts[:3],train_labels[:3]):
  print(encode_sentence(a),b)

Train Dataset Encode Examples
[195  37 196  85 197   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 0
[  2  86  12 198 199  38  29   9  87  15 200 201   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 1
[ 13   4 202  12 203 204   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0] 0


In [40]:
import numpy as np
from sklearn.metrics import accuracy_score

class SimpleNN:
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        # Initialize weights and biases
        self.embedding_weights = np.random.rand(vocab_size, embedding_dim) # Embedding weights
        # limit = np.sqrt(6 / (embedding_dim + hidden_size))
        # self.W1 = np.random.uniform(-limit, limit, size=(embedding_dim, hidden_size))
        # limit = np.sqrt(6/ (hidden_size+output_size))
        # self.W2 = np.random.uniform(-limit, limit, size=(hidden_size, output_size))
        limit = np.sqrt(6 / (embedding_dim + hidden_size))
        self.W1 = np.random.uniform(-limit, limit, size=(embedding_dim, hidden_size))
        limit = np.sqrt(6 / (hidden_size + output_size))
        self.W2 = np.random.uniform(-limit, limit, size=(hidden_size, output_size))
        self.b1 = np.zeros((1, hidden_size))  # Hidden layer biases
        self.b2 = np.zeros((1, output_size))  # Output layer biases
        self.X = None

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_derivative(self, z):
        return z * (1 - z)

    def forward(self, X):
        X = np.asarray(X)
        if X.ndim == 1 and np.issubdtype(X.dtype, np.integer):
            emb = self.embedding_weights[X]
            x = emb.mean(axis=0, keepdims=True)
        else:
            x = X.reshape(1, -1)
        z1 = x @ self.W1 + self.b1
        h  = np.tanh(z1)
        z2 = h @ self.W2 + self.b2
        output = 1.0 / (1.0 + np.exp(-z2))
        output = np.clip(output, 1e-12, 1.0 - 1e-12)
        self.cache = (x, h, output)
        return output


    def compute_loss(self, y, output):
        # Compute binary cross-entropy loss
        y = np.array(y).reshape(output.shape) #
        return -np.sum(y * np.log(output) + (1 - y) * np.log(1 - output))

    def backward(self, X, y, output, learning_rate=0.01):
        x, h, out = self.cache
        dz2 = out - float(y)
        dW2 = h.T @ dz2
        db2 = dz2
        dh  = dz2 @ self.W2.T
        dz1 = dh * (1.0 - h*h)
        dW1 = x.T @ dz1
        db1 = dz1
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1







In [41]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def train(X, y, learning_rate=0.01):
    output = nn.forward(X)  # Forward pass
    loss = nn.compute_loss(y, output)  # Compute loss
    nn.backward(X, y, output, learning_rate)  # Backward pass
    return loss

def predict(x):
    output = nn.forward(x)
    return output, (output > 0.5).astype(int)  # Binary classification

# Initialize the neural network
vocab_size = len(vocab_to_int)  # Number of unique words in vocab
embedding_dim = 100  # Embedding dimension
hidden_size = 80  # Number of neurons in the hidden layer
output_size = 1  # One output (binary classification)
learning_rate = 0.01
nn = SimpleNN(vocab_size, embedding_dim, hidden_size, output_size)

true, pred = [], []
epochs = 100
for epoch in range(epochs):
    train_loss = 0.0
    true, pred = [], []  # Reset true and predicted labels for each epoch
    for x, y in zip(train_texts, train_labels):
        x = encode_sentence(x)  # Encode sentence as word indices
        train_loss += train(x, y, learning_rate)  # Train on the current sample
        _, prediction = predict(x)  # obtain prediction
        true.append(y)  # Append true label
        pred.append(prediction[0][0])  # Append predicted label (extract scalar)

    # Calculate training accuracy
    train_acc = accuracy_score(true, pred) * 100.0
    train_loss /= len(train_texts)  # Average training loss

    # Evaluate on dev set
    dev_true, dev_pred = [], []
    dev_loss = 0.0
    for x, y in zip(dev_texts, dev_labels):
        x = encode_sentence(x)

        output, prediction = predict(x)
        dev_loss += nn.compute_loss(y,output)
        dev_true.append(y)
        dev_pred.append(prediction[0][0])
    dev_loss /= len(dev_texts)
    dev_acc = accuracy_score(dev_true, dev_pred) * 100.0
    print(f'{epoch} epoch, train_loss = {train_loss:.4f}, train_acc: {train_acc:.2f}%, eval_loss: {dev_loss:.4f}, eval_acc: {dev_acc:.2f}%')


0 epoch, train_loss = 0.7486, train_acc: 71.00%, eval_loss: 0.7025, eval_acc: 50.00%
1 epoch, train_loss = 0.7433, train_acc: 76.00%, eval_loss: 0.7027, eval_acc: 50.00%
2 epoch, train_loss = 0.7404, train_acc: 75.00%, eval_loss: 0.7029, eval_acc: 50.00%
3 epoch, train_loss = 0.7377, train_acc: 74.00%, eval_loss: 0.7031, eval_acc: 50.00%
4 epoch, train_loss = 0.7354, train_acc: 74.00%, eval_loss: 0.7034, eval_acc: 50.00%
5 epoch, train_loss = 0.7334, train_acc: 73.00%, eval_loss: 0.7037, eval_acc: 50.00%
6 epoch, train_loss = 0.7315, train_acc: 72.00%, eval_loss: 0.7040, eval_acc: 50.00%
7 epoch, train_loss = 0.7298, train_acc: 72.00%, eval_loss: 0.7043, eval_acc: 50.00%
8 epoch, train_loss = 0.7282, train_acc: 71.00%, eval_loss: 0.7046, eval_acc: 50.00%
9 epoch, train_loss = 0.7268, train_acc: 71.00%, eval_loss: 0.7048, eval_acc: 50.00%
10 epoch, train_loss = 0.7255, train_acc: 71.00%, eval_loss: 0.7051, eval_acc: 50.00%
11 epoch, train_loss = 0.7243, train_acc: 71.00%, eval_loss: 0.7

In [42]:
## NOTE this is test slot for check if the code is working, made by Chat GPT. I saved it just in case, comments are in Finnish

import numpy as np

def pick_nonempty_sample(texts, encode_sentence):
    for t in texts:
        x = np.asarray(encode_sentence(t))
        if x.size > 0:
            return x
    return np.array([0], dtype=int)

def check_hw1(nn, encode_sentence, train_texts, train_labels):
    print("== HW1 forward/backward -itse­tarkistus ==")

    # 1) Muodot ja perusasetukset
    ok = True
    try:
        assert getattr(nn, "W2").shape[1] == 1, "W2.shape[1] pitää olla 1 (binääriulostulo)"
        assert getattr(nn, "b2").shape == (1,1), "b2.shape pitää olla (1,1)"
    except AssertionError as e:
        ok = False
        print("✗", e)
    else:
        print("✓ Ulostulokerroksen muodot ok (W2: *,1 ja b2: 1,1)")

    # 2) Forward antaa järkevän todennäköisyyden
    xi = pick_nonempty_sample(train_texts, encode_sentence)
    yi = float(train_labels[0])

    out = nn.forward(xi)
    out_val = float(np.squeeze(out))
    if out.shape == (1,1) and 0.0 < out_val < 1.0:
        print("✓ forward: output.shape == (1,1) ja 0<output<1")
    else:
        ok = False
        print(f"✗ forward: odotin (1,1) ja arvo 0–1, sain shape={out.shape}, val={out_val}")

    # 3) Loss ei ole NaN/Inf
    loss0 = nn.compute_loss(yi, out)
    if np.isfinite(loss0):
        print(f"✓ compute_loss finite (loss0={float(loss0):.6f})")
    else:
        ok = False
        print("✗ compute_loss antoi NaN/Inf → klippaa output tai lisää eps")

    # 4) Backward päivittää vain W1,b1,W2,b2 (ei embeddingiä) ja loss laskee
    W1o, b1o = nn.W1.copy(), nn.b1.copy()
    W2o, b2o = nn.W2.copy(), nn.b2.copy()
    Eo = nn.embedding_weights.copy()

    nn.backward(xi, yi, out, learning_rate=0.01)

    changed = {
        "W1": np.any(nn.W1 != W1o),
        "b1": np.any(nn.b1 != b1o),
        "W2": np.any(nn.W2 != W2o),
        "b2": np.any(nn.b2 != b2o),
        "Emb": np.any(nn.embedding_weights != Eo),
    }
    print("Päivittyi:", changed)

    if changed["W1"] and changed["b1"] and changed["W2"] and changed["b2"] and (not changed["Emb"]):
        print("✓ Päivitykset vain W1,b1,W2,b2 (embedding ei päivity) ")
    else:
        ok = False
        print("✗ Väärät parametrit päivittyivät (tai embedding päivittyi)")

    # 5) Loss ennen/jälkeen yhden askeleen
    out1 = nn.forward(xi)
    loss1 = nn.compute_loss(yi, out1)
    if np.isfinite(loss1) and float(loss1) <= float(loss0):
        print(f"✓ loss before→after: {float(loss0):.6f} → {float(loss1):.6f}")
    else:
        ok = False
        print(f"✗ loss ei pienentynyt: {float(loss0):.6f} → {float(loss1):.6f}")

    print("== Tulos:", "OK ✅" if ok else "Korjattavaa ❌")
    return ok

check_hw1(nn, encode_sentence, train_texts, train_labels)


== HW1 forward/backward -itse­tarkistus ==
✓ Ulostulokerroksen muodot ok (W2: *,1 ja b2: 1,1)
✓ forward: output.shape == (1,1) ja 0<output<1
✓ compute_loss finite (loss0=0.689307)
Päivittyi: {'W1': np.True_, 'b1': np.True_, 'W2': np.True_, 'b2': np.True_, 'Emb': np.False_}
✓ Päivitykset vain W1,b1,W2,b2 (embedding ei päivity) 
✓ loss before→after: 0.689307 → 0.670111
== Tulos: OK ✅


True